- quality_api.csv
- non_reliable.csv
- reliable_text.csv
- reliable.csv

In [88]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
import requests
import pandas as pd

In [94]:
titles = pd.read_csv('/content/drive/MyDrive/학교/Dissertation/Data Collection2/titles.csv')
titles.drop(['Unnamed: 0'], axis=1, inplace=True)
titles

,title,quality
0,Anarchism,GA
1,Albedo,NaN
2,A,NaN
3,Alabama,NaN
4,Achilles,NaN
...,...,...
138412,Intelligentsia,NaN
138413,Boch Center,NaN
138414,Wang Theatre,NaN
138415,Shubert Theatre,NaN


In [95]:
titles['title'] = titles['title'].astype(str)

In [96]:
def get_wikipedia_article_info(titles):
    # Define the endpoint
    endpoint = "https://en.wikipedia.org/w/api.php"

    # Define parameters
    params = {
        'action': 'query',
        'format': 'json',
        'prop': 'info|pageassessments',
        'titles': '|'.join(titles)
    }

    # Make the request
    response = requests.get(endpoint, params=params)

    # Check if the request was successful
    if response.status_code != 200:
        return f"Error: Unable to fetch data from Wikipedia API. Status code: {response.status_code}"

    # Parse the response
    data = response.json()
    pages = data['query']['pages']

    # Extract relevant information
    article_info = []
    for page_id, page in pages.items():
        title = page.get('title')
        page_id = page.get('pageid')
        quality_class = None
        if 'pageassessments' in page:
            assessments = page['pageassessments']
            if assessments:
                quality_class = next(iter(assessments.values())).get('class')

        article_info.append({
            'title': title,
            'page_id': page_id,
            'quality_class': quality_class
        })

    return article_info

In [97]:
# Example usage
title = ["Alabama", "Steve Grand (roboticist)", "Viscount Grandison"]
article_info = get_wikipedia_article_info(title)
for info in article_info:
    print(info)

{'title': 'Alabama', 'page_id': 303, 'quality_class': 'B'}
{'title': 'Steve Grand (roboticist)', 'page_id': 558376, 'quality_class': 'Start'}
{'title': 'Viscount Grandison', 'page_id': 558387, 'quality_class': None}


In [98]:
quality1 = []
for i in range(0, 20000, 50):
    lst = titles['title'].iloc[i:i+50].tolist()
    df = get_wikipedia_article_info(lst)
    quality1.append(df)

In [99]:
len(quality1)

400

In [100]:
quality2 = []
for i in range(20000, 40000, 50):
    lst = titles['title'].iloc[i:i+50].tolist()
    df = get_wikipedia_article_info(lst)
    quality2.append(df)

In [101]:
len(quality2)

400

In [102]:
quality3 = []
for i in range(40000, 60000, 50):
    lst = titles['title'].iloc[i:i+50].tolist()
    df = get_wikipedia_article_info(lst)
    quality3.append(df)

In [103]:
len(quality3)

400

In [104]:
quality4 = []
for i in range(60000, 80000, 50):
    lst = titles['title'].iloc[i:i+50].tolist()
    df = get_wikipedia_article_info(lst)
    quality4.append(df)

In [105]:
len(quality4)

400

In [106]:
quality5 = []
for i in range(80000, 100000, 50):
    lst = titles['title'].iloc[i:i+50].tolist()
    df = get_wikipedia_article_info(lst)
    quality5.append(df)

In [107]:
len(quality5)

400

In [108]:
quality6 = []
for i in range(100000, 120000, 50):
    lst = titles['title'].iloc[i:i+50].tolist()
    df = get_wikipedia_article_info(lst)
    quality6.append(df)

In [109]:
len(quality6)

400

In [110]:
quality7 = []
for i in range(120000, 138417, 50):
    lst = titles['title'].iloc[i:i+50].tolist()
    df = get_wikipedia_article_info(lst)
    quality7.append(df)

In [111]:
len(quality7)

369

In [112]:
quality = []
lst = [quality1, quality2, quality3, quality4, quality5, quality6]
for i in lst:
    for j in range(400):
        quality.append(i[j])

In [113]:
len(quality)

2400

In [114]:
for i in range(len(quality7)):
    quality.append(quality7[i])

In [115]:
len(quality)

2769

In [116]:
df_quality = pd.DataFrame(quality[0])
for i in range(1, len(quality)):
    df = pd.DataFrame(quality[i])
    df_quality = pd.concat([df_quality, df])

In [117]:
len(df_quality)

138417

In [118]:
df_quality.reset_index(drop=True, inplace=True)

In [119]:
df_quality

,title,page_id,quality_class
0,A,290.0,None
1,ASCII,586.0,None
2,Abacus,655.0,None
3,Abraham Lincoln,307.0,None
4,Academy Award for Best Production Design,316.0,None
...,...,...,...
138412,Stressed Eric,311291.0,None
138413,Thomas Walker (naval officer),311260.0,Start
138414,Treaty of Alcáçovas,311272.0,None
138415,Walter Pater,311303.0,None


In [120]:
df_quality.isnull().sum()

title                 0
page_id               1
quality_class    125325
dtype: int64

In [121]:
df_quality.drop(df_quality[df_quality['page_id'].isnull()].index, inplace=True)
df_quality.reset_index(drop=True, inplace=True)

In [122]:
df_quality

,title,page_id,quality_class
0,A,290.0,None
1,ASCII,586.0,None
2,Abacus,655.0,None
3,Abraham Lincoln,307.0,None
4,Academy Award for Best Production Design,316.0,None
...,...,...,...
138411,Stressed Eric,311291.0,None
138412,Thomas Walker (naval officer),311260.0,Start
138413,Treaty of Alcáçovas,311272.0,None
138414,Walter Pater,311303.0,None


In [123]:
df_quality['page_id'] = df_quality['page_id'].astype(int)

In [124]:
df_quality

,title,page_id,quality_class
0,A,290,None
1,ASCII,586,None
2,Abacus,655,None
3,Abraham Lincoln,307,None
4,Academy Award for Best Production Design,316,None
...,...,...,...
138411,Stressed Eric,311291,None
138412,Thomas Walker (naval officer),311260,Start
138413,Treaty of Alcáçovas,311272,None
138414,Walter Pater,311303,None


In [125]:
df_quality['quality_class'].value_counts()

quality_class
Start       6997
C           2480
B           1248
Stub         822
List         538
             326
GA           248
Disambig     229
FA           114
File          33
FL            16
Redirect      10
SIA           10
A              6
NA             5
Portal         3
list           2
CL             2
Template       1
BL             1
Draft          1
Name: count, dtype: int64

In [126]:
df_quality.to_csv('/content/drive/MyDrive/학교/Dissertation/Data Collection2/quality_api.csv')

### quality class

1. B-class

In [127]:
B = df_quality[df_quality['quality_class']=='B']
B

,title,page_id,quality_class
15,Albedo,39,B
182,Argon,896,B
298,Avicenna,1130,B
308,Alexander Mackenzie (politician),1235,B
373,Americans with Disabilities Act of 1990,1338,B
...,...,...,...
138099,Antoninus Liberalis,310260,B
138183,Pictish language,310437,B
138199,Achewood,310619,B
138217,Fitness landscape,310599,B


2. C-class

In [128]:
C = df_quality[df_quality['quality_class']=='C']
C

,title,page_id,quality_class
74,American National Standards Institute,659,C
148,Mouthwash,782,C
158,Agnosticism,894,C
202,Aaron,1006,C
392,Apple I,1344,C
...,...,...,...
138331,Modern Jazz Quartet,310911,C
138395,Victor Vasarely,311107,C
138399,American Humanist Association,311264,C
138400,Binge eating disorder,311254,C


3. Start-class

In [129]:
Start = df_quality[df_quality['quality_class']=='Start']
Start

,title,page_id,quality_class
131,Anthophyta,779,Start
161,Alfons Maria Jakob,892,Start
305,Acoustic theory,1234,Start
330,Ashmore and Cartier Islands,1227,Start
346,Telecommunications in Anguilla,1223,Start
...,...,...,...
138245,V20 engine,310614,Start
138262,Cookie Gilchrist,310743,Start
138303,Blues Magoos,310895,Start
138377,Neustar,311108,Start


4. Stub-class

In [130]:
Stub = df_quality[df_quality['quality_class']=='Stub']
Stub

,title,page_id,quality_class
454,Agapanthus africanus,1543,Stub
618,Andrea Andreani,1754,Stub
923,Alfonso Leng,2329,Stub
1132,Arthur J. Stone,2729,Stub
1319,Agra Canal,3122,Stub
...,...,...,...
138068,Homework First,310103,Stub
138143,Wilhelm Xylander,310253,Stub
138216,Drive-in,310612,Stub
138397,Winkworth Arboretum,311111,Stub


----------

In [131]:
non_reliable = pd.concat([B, C, Start, Stub])
non_reliable

,title,page_id,quality_class
15,Albedo,39,B
182,Argon,896,B
298,Avicenna,1130,B
308,Alexander Mackenzie (politician),1235,B
373,Americans with Disabilities Act of 1990,1338,B
...,...,...,...
138068,Homework First,310103,Stub
138143,Wilhelm Xylander,310253,Stub
138216,Drive-in,310612,Stub
138397,Winkworth Arboretum,311111,Stub


In [132]:
non_reliable.reset_index(drop=True, inplace=True)

In [133]:
non_reliable.rename(columns={'quality_class': 'quality'}, inplace=True)

In [134]:
non_reliable

,title,page_id,quality
0,Albedo,39,B
1,Argon,896,B
2,Avicenna,1130,B
3,Alexander Mackenzie (politician),1235,B
4,Americans with Disabilities Act of 1990,1338,B
...,...,...,...
11542,Homework First,310103,Stub
11543,Wilhelm Xylander,310253,Stub
11544,Drive-in,310612,Stub
11545,Winkworth Arboretum,311111,Stub


In [135]:
non_reliable.to_csv('/content/drive/MyDrive/학교/Dissertation/Data Collection2/non_reliable.csv')

----------

5. FA & GA

In [136]:
titles_r = pd.read_csv('/content/drive/MyDrive/학교/Dissertation/Data Collection2/titles_r.csv')
titles_r.drop(['Unnamed: 0'], axis=1, inplace=True)
titles_r

,title,quality
0,Anarchism,GA
1,Abraham Lincoln,GA
2,Aristotle,GA
3,Ayn Rand,GA
4,Amphibian,FA
...,...,...
4453,Bat'leth,GA
4454,Starflight,GA
4455,Margaret Fuller,FA
4456,St Pancras railway station,GA


In [139]:
reliable = df_quality[(df_quality['quality_class']=='FA')|(df_quality['quality_class']=='GA')]
reliable.drop(['page_id'], axis=1, inplace=True)
reliable

<ipython-input-139-207b21a2e201>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reliable.drop(['page_id'], axis=1, inplace=True)


,title,quality_class
29,Anarchism,GA
83,Apollo 11,FA
111,Alexander the Great,GA
619,Andrew II of Hungary,GA
842,Astrology,GA
...,...,...
135697,Vic Fontaine,GA
137368,Hoodening,FA
137635,President of Croatia,GA
138300,Bat'leth,GA


In [140]:
titles_r2 = titles_r.merge(reliable, how='outer')
titles_r2.reset_index(drop=True, inplace=True)
titles_r2

,title,quality,quality_class
0,Anarchism,GA,GA
1,Abraham Lincoln,GA,NaN
2,Aristotle,GA,NaN
3,Ayn Rand,GA,NaN
4,Amphibian,FA,NaN
...,...,...,...
4455,Margaret Fuller,FA,NaN
4456,St Pancras railway station,GA,NaN
4457,Water rail,FA,NaN
4458,George Pickett,NaN,GA


In [142]:
def get_wikipedia_articles(titles):
    # Define the endpoint URL
    endpoint = "https://en.wikipedia.org/w/api.php"

    # Prepare the list to store all articles
    all_articles = []

    for title in titles:
        # Define parameters for each title
        params = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "revisions",
            "rvprop": "content",
            "rvslots": "main"
        }

        # Make the request
        response = requests.get(endpoint, params=params)

        # Check if the request was successful
        if response.status_code != 200:
            all_articles.append({
                'title': title,
                'error': f"Error: Unable to fetch data from Wikipedia API. Status code: {response.status_code}"
            })
            continue

        # Parse the response
        data = response.json()
        pages = data['query']['pages']

        # Extract relevant information
        for page_id, page_info in pages.items():
            title = page_info.get('title')
            page_id = page_info.get('pageid')
            content = page_info['revisions'][0]['slots']['main']['*']

            all_articles.append({
                'title': title,
                'page_id': page_id,
                'content': content
            })

    return all_articles

In [143]:
text1 = []
for i in range(0, 2000, 50):
    lst = titles_r2['title'].iloc[i:i+50].tolist()
    df_add = get_wikipedia_articles(lst)
    text1.append(df_add)

In [144]:
len(text1)

40

In [145]:
text2 = []
for i in range(2000, 4000, 50):
    lst = titles_r2['title'].iloc[i:i+50].tolist()
    df_add = get_wikipedia_articles(lst)
    text2.append(df_add)

In [146]:
len(text2)

40

In [147]:
text3 = []
for i in range(4000, 4459, 50):
    lst = titles_r2['title'].iloc[i:i+50].tolist()
    df_add = get_wikipedia_articles(lst)
    text3.append(df_add)

In [148]:
len(text3)

10

In [149]:
text = []
lst = [text1, text2]
for i in lst:
    for j in range(40):
        text.append(i[j])

In [150]:
len(text)

80

In [151]:
for i in range(len(text3)):
    text.append(text3[i])

In [152]:
len(text)

90

In [153]:
df_text_tags = pd.DataFrame(text[0])
for i in range(1, len(text)):
    df_add = pd.DataFrame(text[i])
    df_text_tags = pd.concat([df_text_tags, df_add])

In [154]:
len(df_text_tags)

4460

In [155]:
df_text_tags.isnull().sum()

title      0
page_id    0
content    0
dtype: int64

In [156]:
df_text_tags.reset_index(drop=True, inplace=True)

In [157]:
df_text_tags

,title,page_id,content
0,Anarchism,12,{{Short description|Political philosophy and m...
1,Abraham Lincoln,307,{{Short description|President of the United St...
2,Aristotle,308,{{Short description|Ancient Greek philosopher ...
3,Ayn Rand,339,{{Short description|Russian-born American auth...
4,Amphibian,621,{{Short description|Class of ectothermic tetra...
...,...,...,...
4455,Margaret Fuller,311144,{{short description|American writer and women'...
4456,St Pancras railway station,311178,{{short description|Railway terminus in centra...
4457,Water rail,311236,{{short description|Species of bird}}\n{{Speci...
4458,George Pickett,146929,{{Short description|Confederate army general (...


In [158]:
df_text_tags.to_csv('/content/drive/MyDrive/학교/Dissertation/Data Collection2/reliable_text.csv')

In [159]:
df_text_tags['quality2'] = None
df_text_tags.loc[df_text_tags['content'].str.contains('good article', case=False), 'quality2'] = 'GA'
df_text_tags.loc[df_text_tags['content'].str.contains('featured article', case=False), 'quality2'] = 'FA'

In [160]:
df_text_tags

,title,page_id,content,quality2
0,Anarchism,12,{{Short description|Political philosophy and m...,GA
1,Abraham Lincoln,307,{{Short description|President of the United St...,GA
2,Aristotle,308,{{Short description|Ancient Greek philosopher ...,GA
3,Ayn Rand,339,{{Short description|Russian-born American auth...,GA
4,Amphibian,621,{{Short description|Class of ectothermic tetra...,FA
...,...,...,...,...
4455,Margaret Fuller,311144,{{short description|American writer and women'...,FA
4456,St Pancras railway station,311178,{{short description|Railway terminus in centra...,GA
4457,Water rail,311236,{{short description|Species of bird}}\n{{Speci...,FA
4458,George Pickett,146929,{{Short description|Confederate army general (...,None


In [162]:
df_text_tags['quality2'].value_counts()

quality2
GA    3135
FA    1317
Name: count, dtype: int64

In [165]:
df_text_tags[df_text_tags['quality2'].isnull()]

,title,page_id,content,quality2
140,Binary search algorithm,77111278,#REDIRECT [[Binary search]]\n\n{{Redirect cate...,None
238,COBOL,6799,{{short description|Programming language with ...,None
790,Polio,25107,{{Short description|Infectious disease caused ...,None
1862,Forbidden City,69123,{{Short description|Imperial palace complex in...,None
1865,Tranmere Rovers F.C.,69219,{{Short description|Association football club ...,None
3170,The Sims Online,184411,{{short description|2002 massively multiplayer...,None
3860,Florida State University,239297,{{Short description|Public university in Talla...,None
4458,George Pickett,146929,{{Short description|Confederate army general (...,None


In [166]:
df_text_tags.drop(df_text_tags[df_text_tags['quality2'].isnull()].index, inplace=True)
df_text_tags.reset_index(drop=True, inplace=True)
df_text_tags

,title,page_id,content,quality2
0,Anarchism,12,{{Short description|Political philosophy and m...,GA
1,Abraham Lincoln,307,{{Short description|President of the United St...,GA
2,Aristotle,308,{{Short description|Ancient Greek philosopher ...,GA
3,Ayn Rand,339,{{Short description|Russian-born American auth...,GA
4,Amphibian,621,{{Short description|Class of ectothermic tetra...,FA
...,...,...,...,...
4447,Starflight,310917,{{Short description|1986 video game}}\n{{About...,GA
4448,Margaret Fuller,311144,{{short description|American writer and women'...,FA
4449,St Pancras railway station,311178,{{short description|Railway terminus in centra...,GA
4450,Water rail,311236,{{short description|Species of bird}}\n{{Speci...,FA


In [167]:
df_text_tags.rename(columns={'quality2': 'quality'}, inplace=True)

In [168]:
df_text_tags

,title,page_id,content,quality
0,Anarchism,12,{{Short description|Political philosophy and m...,GA
1,Abraham Lincoln,307,{{Short description|President of the United St...,GA
2,Aristotle,308,{{Short description|Ancient Greek philosopher ...,GA
3,Ayn Rand,339,{{Short description|Russian-born American auth...,GA
4,Amphibian,621,{{Short description|Class of ectothermic tetra...,FA
...,...,...,...,...
4447,Starflight,310917,{{Short description|1986 video game}}\n{{About...,GA
4448,Margaret Fuller,311144,{{short description|American writer and women'...,FA
4449,St Pancras railway station,311178,{{short description|Railway terminus in centra...,GA
4450,Water rail,311236,{{short description|Species of bird}}\n{{Speci...,FA


In [169]:
reliable = df_text_tags[['title', 'page_id', 'quality']]
reliable

,title,page_id,quality
0,Anarchism,12,GA
1,Abraham Lincoln,307,GA
2,Aristotle,308,GA
3,Ayn Rand,339,GA
4,Amphibian,621,FA
...,...,...,...
4447,Starflight,310917,GA
4448,Margaret Fuller,311144,FA
4449,St Pancras railway station,311178,GA
4450,Water rail,311236,FA


In [170]:
reliable.to_csv('/content/drive/MyDrive/학교/Dissertation/Data Collection2/reliable.csv')